In [1]:
from IPython.display import display

import numpy as np
import pandas as pd
import os
from ebmdatalab import bq
import datetime
import plotly.express as px
import plotly.io as pio
from IPython.display import Markdown
import warnings
import requests
warnings.filterwarnings('ignore')
pio.templates.default = "plotly_white"
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from pandas.plotting import register_matplotlib_converters
import scipy.stats as stats
register_matplotlib_converters()
from statsforecast import StatsForecast

from statsforecast.models import ARIMA
from time import time
import seaborn as sns
sns.set(style="whitegrid")
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from statsforecast.arima import arima_string
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = np.random.seed(0)


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
dark_style = {
    'figure.facecolor': '#212946',
    'axes.facecolor': '#212946',
    'savefig.facecolor':'#212946',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#2A3459',
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 }
plt.rcParams.update(dark_style)

from pylab import rcParams
rcParams['figure.figsize'] = (18,7)

### Get the data from BQ

In [116]:
sql = """
SELECT ampp.id as ampp, ampp.nm, ampp.bnf_code, priceinfo_price, priceinfo_pricedt, priceinfo_price_prev, priceinfo_price_basis, qtyval, qty_uom  FROM `ebmdatalab.dmd.ampp_full`  as ampp
inner join
dmd.vmpp_full as vmpp
on
ampp.vmpp = vmpp.id
where priceinfo_price_basis = 'NHS Indicative Price'
"""
exportfile = os.path.join("..","data","amp_price_df.csv") #defines name for cache file

Downloading: 100%|█████████████████████████████████████████████████████████████|


In [117]:
amp_price_df.head()

,ampp,nm,bnf_code,priceinfo_price,priceinfo_pricedt,priceinfo_price_prev,priceinfo_price_basis,qtyval,qty_uom
0,1327911000001107,Verapamil 160mg tablets (A A H Pharmaceuticals...,0206020T0AAAGAG,4862,2024-08-05,4420,NHS Indicative Price,56.000000000,tablet
1,1328111000001105,Verapamil 160mg tablets (Accord-UK Ltd) 56 tab...,0206020T0AAAGAG,3153,2021-03-02,2947,NHS Indicative Price,56.000000000,tablet
2,1328211000001104,Zocor 10mg tablets (Organon Pharma (UK) Ltd) 2...,0212000Y0BBAAAA,1803,NaT,<NA>,NHS Indicative Price,28.000000000,tablet
3,1328311000001107,Zocor 20mg tablets (Organon Pharma (UK) Ltd) 2...,0212000Y0BBABAB,2969,NaT,<NA>,NHS Indicative Price,28.000000000,tablet
4,1328411000001100,Zocor 40mg tablets (Organon Pharma (UK) Ltd) 2...,0212000Y0BBACAD,2969,NaT,<NA>,NHS Indicative Price,28.000000000,tablet


In [119]:
# Convert priceinfo_pricedt to datetime
amp_price_df['priceinfo_pricedt'] = pd.to_datetime(amp_price_df['priceinfo_pricedt'])

In [74]:

# Determine the minimum and maximum dates
min_date = '2020-04-01' #(set as variable at some point)
max_date = '2024-06-01'


In [124]:
df = amp_price_df

# Define the range of months you want to include
start_date = '2020-06-01'
end_date = '2024-08-01'
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')

expanded_df = pd.DataFrame(date_range, columns=['month'])


expanded_df = expanded_df.loc[expanded_df.index.repeat(len(df))].reset_index(drop=True)
original_data = pd.concat([df]*len(date_range), ignore_index=True).reset_index(drop=True)

result_df = pd.concat([expanded_df, original_data], axis=1)

def calculate_price_new(row):
    # Check if 'pricedt' is NaT or if 'month' is greater than or equal to 'pricedt'
    if pd.isna(row['priceinfo_pricedt']) or row['month'] >= row['priceinfo_pricedt']:
        return row['priceinfo_price']
    else:
        return row['priceinfo_price_prev']

# Apply the function to create the 'price_new' column
result_df['price_new'] = result_df.apply(calculate_price_new, axis=1)
result_df['price_per_unit'] = result_df['price_new']/result_df['qtyval']
result_df.drop(columns=['priceinfo_price', 'priceinfo_pricedt','priceinfo_price_prev', 'priceinfo_price_basis' ], inplace=True)



In [ ]:
import pandas as pd

# Sample DataFrame
data = {
    'ampp': [1, 2],
    'price': [100, 200],
    'price_prev': [90, 180],
    'price_basis': [95, 190]
}

df = pd.DataFrame(data)

# Define the range of months you want to include
start_date = '2024-01-01'
end_date = '2024-12-01'
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')

# Create a new DataFrame with the full date range
expanded_df = pd.DataFrame(date_range, columns=['month'])

# Now expand the original DataFrame to match the new date range
# Here we're assuming that each original entry should be repeated for every month in the range.
# If you have specific data to match, adjust accordingly.

# Repeat the original data for each month
expanded_df = expanded_df.loc[expanded_df.index.repeat(len(df))].reset_index(drop=True)
original_data = pd.concat([df]*len(date_range), ignore_index=True).reset_index(drop=True)

# Merge the expanded DataFrame with the original DataFrame
result_df = pd.concat([expanded_df, original_data], axis=1)

print(result_df)

In [1]:
result_df.head(20)

NameError: name 'result_df' is not defined

In [126]:
result_df['row_number'] = result_df.groupby(['month', 'bnf_code'])['price_per_unit'].rank(method='first', ascending=False)


In [138]:
filtered_amp_df = result_df[result_df['row_number'] == 1]

In [139]:
filtered_amp_df.head()

,month,ampp,nm,bnf_code,qtyval,qty_uom,price_new,price_per_unit,row_number
0,2020-06-01,1327911000001107,Verapamil 160mg tablets (A A H Pharmaceuticals...,0206020T0AAAGAG,56.000000000,tablet,4420,78.92857142857142857142857143,1.0
2,2020-06-01,1328211000001104,Zocor 10mg tablets (Organon Pharma (UK) Ltd) 2...,0212000Y0BBAAAA,28.000000000,tablet,1803,64.39285714285714285714285714,1.0
3,2020-06-01,1328311000001107,Zocor 20mg tablets (Organon Pharma (UK) Ltd) 2...,0212000Y0BBABAB,28.000000000,tablet,2969,106.0357142857142857142857143,1.0
4,2020-06-01,1328411000001100,Zocor 40mg tablets (Organon Pharma (UK) Ltd) 2...,0212000Y0BBACAD,28.000000000,tablet,2969,106.0357142857142857142857143,1.0
8,2020-06-01,1330811000001104,Tenormin LS 50mg tablets (Atnahs Pharma UK Ltd...,0204000E0BBAFAB,28.000000000,tablet,1022,36.5,1.0


In [129]:
sql = """
SELECT * FROM dmd.tariffprice as dt
inner join
dmd.vmpp as vmpp
on
dt.vmpp = vmpp.id
"""
exportfile = os.path.join("..","data","dt_price_df.csv") #defines name for cache file
dt_price_df = bq.cached_read(sql, csv_path=exportfile, use_cache=True) #uses BQ if changed, otherwise csv cache file

Downloading: 100%|█████████████████████████████████████████████████████████████|


In [135]:
dt_price_df.head()

,date,vmpp,tariff_category,price_pence,id,invalid,nm,vmp,qtyval,qty_uom,combpack,bnf_code,price_per_unit,row_number
0,2010-04-01,4302611000001100,1,81,4302611000001100,False,Menthol crystals 5 gram,4317211000001102,5.000000000,258682000,<NA>,190400000AAAIAI,16.2,1.0
1,2010-05-01,4302611000001100,1,81,4302611000001100,False,Menthol crystals 5 gram,4317211000001102,5.000000000,258682000,<NA>,190400000AAAIAI,16.2,1.0
2,2010-06-01,4302611000001100,1,82,4302611000001100,False,Menthol crystals 5 gram,4317211000001102,5.000000000,258682000,<NA>,190400000AAAIAI,16.4,1.0
3,2010-07-01,4302611000001100,1,82,4302611000001100,False,Menthol crystals 5 gram,4317211000001102,5.000000000,258682000,<NA>,190400000AAAIAI,16.4,1.0
4,2010-08-01,4302611000001100,1,83,4302611000001100,False,Menthol crystals 5 gram,4317211000001102,5.000000000,258682000,<NA>,190400000AAAIAI,16.6,1.0


In [131]:
dt_price_df['price_per_unit'] = dt_price_df['price_pence']/dt_price_df['qtyval']

In [134]:
dt_price_df['row_number'] = dt_price_df.groupby(['date', 'bnf_code'])['price_per_unit'].rank(method='first', ascending=False)

In [136]:
filtered_dt_df = dt_price_df[dt_price_df['row_number'] == 1]

In [137]:
filtered_dt_df.head()

,date,vmpp,tariff_category,price_pence,id,invalid,nm,vmp,qtyval,qty_uom,combpack,bnf_code,price_per_unit,row_number
0,2010-04-01,4302611000001100,1,81,4302611000001100,False,Menthol crystals 5 gram,4317211000001102,5.000000000,258682000,<NA>,190400000AAAIAI,16.2,1.0
1,2010-05-01,4302611000001100,1,81,4302611000001100,False,Menthol crystals 5 gram,4317211000001102,5.000000000,258682000,<NA>,190400000AAAIAI,16.2,1.0
2,2010-06-01,4302611000001100,1,82,4302611000001100,False,Menthol crystals 5 gram,4317211000001102,5.000000000,258682000,<NA>,190400000AAAIAI,16.4,1.0
3,2010-07-01,4302611000001100,1,82,4302611000001100,False,Menthol crystals 5 gram,4317211000001102,5.000000000,258682000,<NA>,190400000AAAIAI,16.4,1.0
4,2010-08-01,4302611000001100,1,83,4302611000001100,False,Menthol crystals 5 gram,4317211000001102,5.000000000,258682000,<NA>,190400000AAAIAI,16.6,1.0


In [170]:
amp_final_df = filtered_amp_df[['month', 'bnf_code', 'price_per_unit']].copy()

In [171]:
dt_final_df = filtered_dt_df[['date', 'bnf_code', 'price_per_unit']].copy()
dt_final_df.rename(columns={"date": "month"}, inplace=True)
dt_final_df['month'] = pd.to_datetime(dt_final_df['month'])

In [172]:
amp_final_df.head()

,month,bnf_code,price_per_unit
0,2020-06-01,0206020T0AAAGAG,78.92857142857142857142857143
2,2020-06-01,0212000Y0BBAAAA,64.39285714285714285714285714
3,2020-06-01,0212000Y0BBABAB,106.0357142857142857142857143
4,2020-06-01,0212000Y0BBACAD,106.0357142857142857142857143
8,2020-06-01,0204000E0BBAFAB,36.5


In [173]:
dt_final_df.head()

,month,bnf_code,price_per_unit
0,2010-04-01,190400000AAAIAI,16.2
1,2010-05-01,190400000AAAIAI,16.2
2,2010-06-01,190400000AAAIAI,16.4
3,2010-07-01,190400000AAAIAI,16.4
4,2010-08-01,190400000AAAIAI,16.6


In [178]:
df_merged = amp_final_df.merge(dt_final_df[['month', 'bnf_code']], on=['month', 'bnf_code'], how='left', indicator=True)

In [179]:
df_merged.head()

,month,bnf_code,price_per_unit,_merge
0,2020-06-01,0206020T0AAAGAG,78.92857142857142857142857143,both
1,2020-06-01,0212000Y0BBAAAA,64.39285714285714285714285714,left_only
2,2020-06-01,0212000Y0BBABAB,106.0357142857142857142857143,left_only
3,2020-06-01,0212000Y0BBACAD,106.0357142857142857142857143,left_only
4,2020-06-01,0204000E0BBAFAB,36.5,left_only


In [180]:
# Step 2: Filter only the rows that are in df2 but not in df1
df_unique = df_merged[df_merged['_merge'] == 'left_only'].drop(columns=['_merge'])

In [181]:
df_unique.head()

,month,bnf_code,price_per_unit
1,2020-06-01,0212000Y0BBAAAA,64.39285714285714285714285714
2,2020-06-01,0212000Y0BBABAB,106.0357142857142857142857143
3,2020-06-01,0212000Y0BBACAD,106.0357142857142857142857143
4,2020-06-01,0204000E0BBAFAB,36.5
8,2020-06-01,0206020T0BDAGAU,27.53571428571428571428571429


In [182]:
df_final = pd.concat([dt_final_df, df_unique], ignore_index=True)

In [183]:
df_final.head()

,month,bnf_code,price_per_unit
0,2010-04-01,190400000AAAIAI,16.2
1,2010-05-01,190400000AAAIAI,16.2
2,2010-06-01,190400000AAAIAI,16.4
3,2010-07-01,190400000AAAIAI,16.4
4,2010-08-01,190400000AAAIAI,16.6
